# Using DBSCAN as clustering method

In [2]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from scipy.sparse import csr_matrix
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs

## Test

In [3]:
states = ["INITIAL","login","View_Items","home","logout","View_Items_quantity","Add_to_Cart","shoppingcart",
          "remove","deferorder","purchasecart","inventory","sellinventory","clearcart","cancelorder"]
sessions = {'HZKS0-WG8pZr0eCsZlBAP5Xm': ['INITIAL','login',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'home',
   'logout','$'],'5tPgZbHdK2Zp+heFBs8HsMkx': ['INITIAL','login',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'deferorder',
   'home',
   'logout','$']}

### Makovchain & sparse matrix

In [94]:
def transition_matrix(sessions, states):
    markovchains = []
    for key, value in sessions.items():
        # labelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(value)
        transformed_s = le.transform(value)
        
        #factorize
        factorize = pd.factorize(value)[0]
        
        # matrix
        n = 1 + max(factorize)  # number of states
        M = [[0] * n for _ in range(n)]

        for (i, j) in zip(factorize, factorize[1:]):
            M[i][j] += 1
        
        # now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f / s for f in row]
                
        # print Matrix style
        #for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
        
        # unique array in the right order
        value = np.array(value)
        _, idx = np.unique(value, return_index=True)
        
        df = pd.DataFrame(data = M, index=value[np.sort(idx)],
                          columns=value[np.sort(idx)])
        df_1 = pd.DataFrame(index=states, columns=states, dtype='float64')
        merge = df_1.merge(df, how='right').fillna(0).round(2).set_index(value[np.sort(idx)])
        
        # csr sparse matrix
        #print(merge)
        merge = csr_matrix(merge.values)#.toarray()
        markovchains.append(merge)
        
    return markovchains

        
#test:
m = transition_matrix(sessions, states)
print(m)

(6, 16)
(10, 16)
[<6x16 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>, <10x16 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>]


In [82]:
X = m
clustering = DBSCAN(eps=3, min_samples=1).fit(X)
labels = clustering.labels_
print(labels)
print(clustering)

[0 0 0 0 0 0]
DBSCAN(algorithm='auto', eps=3, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)


## DBSCAN
Testing DBSCAN with random data.

In [91]:
j = 0
for i in m:
    X = m[j]
    clustering = DBSCAN(eps=2, min_samples=1).fit(X)
    labels = clustering.labels_
    print(labels)
    print(clustering)
    j += 1

[0 0 0 0 0 0]
DBSCAN(algorithm='auto', eps=2, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=1, n_jobs=None, p=None)
[0 0 0 0 0 0 0 0 0 0]
DBSCAN(algorithm='auto', eps=2, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=1, n_jobs=None, p=None)


In [62]:
X = np.array([[1, 2], [2, 2], [2, 3],
              [8, 7], [8, 8], [25, 80]])
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
print(clustering.labels_)

clustering 

[ 0  0  0  1  1 -1]


DBSCAN(algorithm='auto', eps=3, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)

## To do: 
Choosing a metric for calculating the distances.

In [10]:
# Plot result
import pylab as pl
from itertools import cycle

pl.close('all')
pl.figure(1)
pl.clf()

# Black removed and is used for noise instead.
colors = cycle('bgrcmybgrcmybgrcmybgrcmy')
for k, col in zip(set(labels), colors):
    if k == -1:
        # Black used for noise.
        col = 'k'
        markersize = 6
    class_members = [index[0] for index in np.argwhere(labels == k)]
    cluster_core_samples = [index for index in core_samples
                            if labels[index] == k]
    for index in class_members:
        x = X[index]
        if index in core_samples and k != -1:
            markersize = 14
        else:
            markersize = 6
        pl.plot(x[0], x[1], 'o', markerfacecolor=col,
                markeredgecolor='k', markersize=markersize)

pl.title('Estimated number of clusters: %d' % n_clusters_)
pl.show()

NameError: name 'core_samples' is not defined

<Figure size 432x288 with 0 Axes>